# Instamatic

Instamatic is a tool for automated electron diffraction data collection. It has interfaces for interfacing with the TEM (JEOL/TFS) and several cameras (Gatan/ASI Timepix/TVIPS).

https://github.com/stefsmeets/instamatic

This notebook shows how to set up a grid montage

First, we must import and initialize the controller object, which defines the interface to the TEM and the camera. For this example, the microscope interaction is simulated.

In [ ]:
from instamatic import TEMController
from instamatic.formats import write_tiff

ctrl = TEMController.initialize()

Next, we will set the image conditions. We want to collect a montage over 5 by 5 points using the `lowmag` mode at a 100x magnification

In [ ]:
ctrl.mode = "lowmag"
ctrl.magnification.value = 100
grid_x, grid_y = 5, 5

## Setting up the grid montage

The grid montage can be set up as follows:

In [ ]:
from instamatic.montage import *

gm = ctrl.montage()
pos = gm.setup(grid_x, grid_y)

print()
print(gm.stagecoords)

Internally, the acquisition is done using the `ctrl.acquire_at_points` function. Acquisition is started using `gm.start`. Afterwards, the data are saved to a new experiment directory.

In [ ]:
buffer = gm.start()

Finally, we use `gm.to_montage` to give us a `Montage` object, which can be used to stitch the images together.

In [ ]:
m = gm.to_montage()

# unoptimized coords
coords = m.get_montage_coords()

# stitch image, use binning 4 for speed-up and memory conservation
stitched = m.stitch(coords, binning=4, plot=True)

# get coords optimized using cross correlation
dv = m.get_difference_vectors()
coords2 = m.get_optimized_montage_coords(dv)

stitched2 = m.stitch(coords2, binning=4, plot=True)